In [ ]:
!docker rm -f sagemaker_inference_server_app 2>/dev/null || true

config_file = "/workspace/D-FINE/configs/dfine/objects365/dfine_hgnetv2_x_obj2coco.yml"
checkpoint_file = "/workspace/dfine_checkpoints/dfine_x_obj2coco.pth"

IMAGE_NAME = "sagemaker_inference_server"
!cd ..; docker build -t {IMAGE_NAME} .

import os
CONTAINER_NAME = f"{IMAGE_NAME}_app"

host_dump_dir = os.path.join(os.getcwd(), "dump")

docker_cmd = f"""
docker run --gpus=all --rm -m 8g -p 8080:8080 \\
  -e CONFIG_FILE={config_file} \\
  -e CHECKPOINT_FILE={checkpoint_file} \\
  -e DEVICE=cuda:0 \\
  -v "{host_dump_dir}:/dump" \\
  --name {CONTAINER_NAME} \\
  {IMAGE_NAME}
"""

print("Running Docker command:\n", docker_cmd)
# !{docker_cmd}

In [ ]:
import requests, json

response = requests.get("http://localhost:8080/models")
print(json.dumps(response.json(), indent=2))


In [ ]:
import test_utils
import importlib
importlib.reload(test_utils)
from test_utils import get_random_coco_images, get_plate_images, get_local_images, make_local_request, draw_results_by_model

params = {
    "license_plate_recognition": {
        "threshold": 0.5,
        "ocr_confidence_threshold": 0.7
    },
    "object_detection_then_lpr": {
        "threshold": 0.5,
        "classes_to_detect": ["person", "car", "motorcycle", "bus", "truck", "bicycle"],
        "chained_models": [
            {"threshold": 0.6, "ocr_confidence_threshold": 0.7}
        ]
    }
}

test_image = "images/car_far.png"
res = make_local_request(test_image, ["license_plate_recognition", "object_detection_then_lpr"], params=params)
draw_results_by_model(test_image, res)
print(json.dumps(res, indent=2))


In [ ]:
test_image = "images/car_medium.png"
res = make_local_request(test_image, ["license_plate_recognition", "object_detection_and_lpr"], 
                                    classes_to_detect=["person", "car", "motorcycle", "bus", "truck", "bicycle"])
draw_results_by_model(test_image, res)


In [ ]:
test_image = "images/car_far.png"
res = make_local_request(test_image, ["license_plate_recognition", "object_detection_and_lpr"], 
                                    classes_to_detect=["person", "car", "motorcycle", "bus", "truck", "bicycle"])
draw_results_by_model(test_image, res)
print(json.dumps(res, indent=2))

In [ ]:

test_image = "images/people_and_car_front_camera.png"
res = make_local_request(test_image, ["license_plate_recognition", "object_detection_and_lpr", "object_detection_and_ppe"], 
                                    classes_to_detect=["person", "car", "motorcycle", "bus", "truck", "bicycle"])
print(res)
draw_results_by_model(test_image, res)
